In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal, ogr, osr
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon,box
from shapely.geometry import mapping, shape
import cv2
import json
import geopandas as gpd
import numpy as np               # numerical array manipulation
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
import PIL.ImageDraw
import shutil
from itertools import product

import sys
from core.UNet import UNet
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision,F1_score
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.visualize import display_images

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
import tensorflow as tf
print(tf.__version__)

2024-01-03 09:15:38.855901: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA RTX A6000, compute capability 8.6


2024-01-03 09:15:40.371555: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-03 09:15:40.372509: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-01-03 09:15:40.397431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-03 09:15:40.397465: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-03 09:15:40.399884: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-01-03 09:15:40.399959: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-0

2.4.1


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2024-01-03 09:15:41.441840: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 09:15:41.443723: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-03 09:15:41.444092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-03 09:15:41.444138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-03 09:15:41.444183: I tensorflow/stream_executor/platform/default/dso_loader.cc

In [3]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis_bands
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis_bands.Configuration()

self.trained_model_path: /home/nkd/hbh/saved_models/lakes_20240101-2240_AdaDelta_dice_loss_012345_576.h5


In [4]:
# testArea = gps.read_file(r'D:\lakemapping\predition\test\test_area.shp')
# image_dir=r'G:\sample_img_3'
# output_dir=config.input_image_dir
# for root, dirs, files in os.walk(image_dir):
#         for file in files:
#             if file.endswith('.tif') and file.startswith('ndwi'):
#                 ndwiImg = rasterio.open(os.path.join(root, file))
#                 bboxImg = box(*ndwiImg.bounds)
#                 for i in tqdm(testArea.index):
#                     if(bboxImg.intersects(testArea.loc[i]['geometry'])):
#                         oldNDWI=os.path.join(root, file)
#                         newNDWI=os.path.join(output_dir,file)
#                         shutil.copy(oldNDWI, newNDWI)
#                         oldBands=oldNDWI.replace('ndwi_int8','bands_int16')
#                         newBands=newNDWI.replace('ndwi_int8','bands_int16')
#                         shutil.copy(oldBands, newBands)

In [5]:
# Load a pretrained model 
OPTIMIZER = adaDelta
# OPTIMIZER = adam
OPTIMIZER=mixed_precision.LossScaleOptimizer(OPTIMIZER)
model = load_model(config.trained_model_path, custom_objects={'dice loss': dice_loss, 'accuracy':accuracy ,'recall':recall, 'F1_score':F1_score,'precision':precision,'IoU': IoU}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[dice_loss, accuracy,recall, precision, IoU])

2024-01-03 09:15:43.116165: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-03 09:15:43.116536: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-03 09:15:43.116579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-03 09:15:43.116611: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-01-03 09:15:43.116618: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-01-03 09:15:43.116626: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024

In [6]:
# Methods to add results of a patch   to    the total results of a larger area. 
#The operator could be min (useful if there are too many false positives), max (useful for tackle false negatives)
#res:mask [rows,cols] predition=np.squeeze(prediction[i], axis = -1) (col, row, wi, he) = batch_pos[i]
def addTOResult(res, prediction, row, col, he, wi,operator,ie_width):
    currValue = res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width]
    newPredictions = prediction[ie_width:he-ie_width, ie_width:wi-ie_width]
    
# IMPORTANT: MIN can't be used as long as the mask is initialed with 0!!!!! 
#If you want to use MIN initial the mask with -1 and handle the case of default value(-1) separately.
    if operator == 'min': # Takes the min of current prediction and new prediction for each pixel  
        currValue [currValue == -1] = 1 #Replace -1 with 1 in case of MIN  
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.minimum(currValue, newPredictions)
    elif operator == 'max':
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.maximum(currValue, newPredictions)
    else:
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = newPredictions  
    return (res)

In [7]:
# Methods that actually makes the predictions
def predict_using_model(model, batch, batch_pos, mask,operator,ie_width):
#     print(batch)
    tm = np.stack(batch, axis = 0)
#     print(tm)
    prediction = model.predict(tm)
    for i in range(len(batch_pos)): 
        (col, row, wi, he) = batch_pos[i]
        p = np.squeeze(prediction[i], axis = -1)
        # Instead of replacing the current values with new values, use the user specified operator (MIN,MAX,REPLACE)
        mask = addTOResult(mask, p, row, col, he, wi,operator,ie_width)  
    return mask

def detect_lake(fullPath, NDWI_img, bands_img, operator ,width=512, height=512, stride = 256,ie_width=0,bandNum=1):
    nols, nrows = NDWI_img.meta['width'], NDWI_img.meta['height']
    meta = NDWI_img.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current NDWI_img',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, NDWI_img.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, bandNum),-1.0)#Add -1 padding in case of corner images
        if bandNum==1:
            read_ndwi_img=NDWI_img.read(window=window)
            temp_im = read_ndwi_img
        elif bandNum==3:
            read_ndwi_img=NDWI_img.read(window=window)/100
            read_bands_img = bands_img.read(window=window)/1000
            temp_im = np.concatenate((read_ndwi_img,np.expand_dims(read_bands_img[1],axis=0),np.expand_dims(read_bands_img[3],axis=0)), axis=0)
        else:#band==5
            read_ndwi_img=NDWI_img.read(window=window)/100
            read_bands_img = bands_img.read(window=window)/1000
            temp_im = np.concatenate((read_ndwi_img,read_bands_img), axis=0)
        temp_im =  np.transpose(temp_im, axes=(1,2,0))

        patch[:window.height, :window.width] = temp_im   
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
        batch = []
        batch_pos = []

    return(mask, meta)

In [8]:
def raster2vector(raster_path, vecter_path, field_name="class", ignore_values = None):
    
    # 读取路径中的栅格数据
    raster = gdal.Open(raster_path)
    # in_band 为想要转为矢量的波段,一般需要进行转矢量的栅格都是单波段分类结果
    # 若栅格为多波段,需要提前转换为单波段
    band = raster.GetRasterBand(1)
    
    # 读取栅格的投影信息,为后面生成的矢量赋予相同的投影信息
    prj = osr.SpatialReference()
    prj.ImportFromWkt(raster.GetProjection())
    
    
    drv = ogr.GetDriverByName("ESRI Shapefile")
    # 若文件已经存在,删除
    if os.path.exists(vecter_path):
        drv.DeleteDataSource(vecter_path)
    # 创建目标文件
    polygon = drv.CreateDataSource(vecter_path)
    # 创建面图层
    poly_layer = polygon.CreateLayer(vecter_path[:-4], srs=prj, geom_type=ogr.wkbMultiPolygon)
    # 添加浮点型字段,用来存储栅格的像素值
    field = ogr.FieldDefn(field_name, ogr.OFTReal)  
    poly_layer.CreateField(field)
    
    # FPolygonize将每个像元转成一个矩形，然后将相似的像元进行合并
    # 设置矢量图层中保存像元值的字段序号为0
    gdal.FPolygonize(band, None, poly_layer, 0)
    
    # 删除ignore_value链表中的类别要素
    if ignore_values is not None:
        for feature in poly_layer:
            class_value = feature.GetField('class')
            for ignore_value in ignore_values:
                if class_value==ignore_value:
                    # 通过FID删除要素
                    poly_layer.DeleteFeature(feature.GetFID())
                    break
                
    polygon.SyncToDisk()
    polygon = None
    print('Vector File Exported Successfully!')
        
def writeMaskToDisk(detected_mask, detected_meta, wp, write_as_type = 'uint8', th = 0.5, create_countors = False):
    # Convert to correct required before writing
    if 'float' in str(detected_meta['dtype']) and 'int' in write_as_type:
        print(f'Converting prediction from {detected_meta["dtype"]} to {write_as_type}, using threshold of {th}')#float32 to uint8
#         initial code have problem of big lake
        detected_mask[detected_mask<th]=0
        detected_mask[detected_mask>=th]=1
        detected_mask = detected_mask.astype(write_as_type)#'uint8'
        detected_meta['dtype'] =  write_as_type
    
    # compress tif
    detected_meta.update({"compress": 'lzw'})
    
    with rasterio.open(wp, 'w', **detected_meta) as outds:
        outds.write(detected_mask, 1)

In [9]:
# # Predict trees in the all the files in the input image dir 
# # Depending upon the available RAM, images may not to be split before running this cell.
# # Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
# all_files = []
# for root, dirs, files in os.walk(config.input_image_dir):
#     for file in files:
#         if file.endswith(config.input_image_type)and file.startswith(config.ndwi_fn_st):
#              all_files.append((os.path.join(root, file), file))
# print(all_files)

# operator='max'
# suffix='_iew{}'.format(config.ignore_edge_width)
# for fullPath, filename in all_files:
#     outputFile = os.path.join(config.output_dir, filename.replace(config.ndwi_fn_st, config.output_prefix + operator + suffix) )
#     if not os.path.isfile(outputFile) or config.overwrite_analysed_files: 
#         with rasterio.open(fullPath) as NDWI:
#             with rasterio.open(fullPath.replace(config.ndwi_fn_st,config.bands_fn_st).replace('ndwi_int8','bands_int16')) as bands:
#                 print(fullPath)
#                 detectedMask, detectedMeta = detect_lake(fullPath,NDWI,bands,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE,ie_width=config.ignore_edge_width,bandNum=config.band_num)
#                 writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
# #                 vecter_path =outputFile.replace('tif','shp')
# #                 field_name = "class"
# # #            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
# #                 ignore_values = [0]
# #                 raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
#     else:
#         print('File already analysed!', fullPath)
        
# print('finish')

In [10]:
def detect_lake(fullPath, image, operator ,width=512, height=512, stride = 256,ie_width=0,bandNum=1):
    nols, nrows = image.meta['width'], image.meta['height']
    meta = image.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    meta['nodata']=None
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current image',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, image.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, bandNum),-1.0)#Add -1 padding in case of corner images
        read_image=image.read(window=window)
        temp_im = read_image/1000
        temp_im =  np.transpose(temp_im, axes=(1,2,0))

        patch[:window.height, :window.width] = temp_im   
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
        batch = []
        batch_pos = []

    return(mask, meta)

In [11]:
# Predict trees in the all the files in the input image dir 
# Depending upon the available RAM, images may not to be split before running this cell.
# Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
all_files = []
for root, dirs, files in os.walk(r'/home/nkd/hbh/patchesReshape/test'):
    for file in files:
        if file.endswith(config.input_image_type)and file.startswith('pad_image'):
             all_files.append((os.path.join(root, file), file))
print(all_files)

operator='max'
suffix='_iew{}'.format(92)
for fullPath, filename in all_files:
    outputFile = os.path.join(r'/home/nkd/hbh/predition/model0101/test', filename.replace('pad_image', config.output_prefix + operator + suffix) )
    if not os.path.isfile(outputFile) or config.overwrite_analysed_files: 
        with rasterio.open(fullPath) as image:
            print(fullPath)
            detectedMask, detectedMeta = detect_lake(fullPath,image,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE,ie_width=config.ignore_edge_width,bandNum=config.band_num)
            writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
            vecter_path =outputFile.replace('tif','shp')
            field_name = "class"
#            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
            ignore_values = [0]
            raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
    else:
        print('File already analysed!', fullPath)
        
print('finish')

[('/home/nkd/hbh/patchesReshape/test/type4/pad_image_8732.tif', 'pad_image_8732.tif'), ('/home/nkd/hbh/patchesReshape/test/type4/pad_image_3560.tif', 'pad_image_3560.tif'), ('/home/nkd/hbh/patchesReshape/test/type4/pad_image_2618.tif', 'pad_image_2618.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_10203.tif', 'pad_image_10203.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_13624.tif', 'pad_image_13624.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_18569.tif', 'pad_image_18569.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_17374.tif', 'pad_image_17374.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_5719.tif', 'pad_image_5719.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_17388.tif', 'pad_image_17388.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_3461.tif', 'pad_image_3461.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/pad_image_3932.tif', 'pad_image_3932.tif'), ('/home/nkd/hbh/patchesReshape/test/type2/p

15it [00:00, 51.54it/s]2024-01-03 09:15:44.679585: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-03 09:15:44.703042: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz
2024-01-03 09:15:45.015160: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2024-01-03 09:15:51.971091: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-03 09:15:51.971959: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: /usr/local/cuda-11.0/bin/ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to 

Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
/home/nkd/hbh/patchesReshape/test/type4/pad_image_3560.tif
the size of current image 1301 1595


20it [00:01, 19.35it/s]
2024-01-03 09:16:59.834832: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-03 09:17:00.090600: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-03 09:17:00.335421: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-03 09:17:00.578051: W tensorf

Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
/home/nkd/hbh/patchesReshape/test/type4/pad_image_2618.tif
the size of current image 866 1030


...

Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
/home/nkd/hbh/patchesReshape/test/type3/pad_image_12926.tif
the size of current image 3286 2903


110it [00:05, 19.15it/s]


Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
/home/nkd/hbh/patchesReshape/test/type3/pad_image_11689.tif
the size of current image 2447 2220


56it [00:02, 18.84it/s]


Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
finish


In [12]:
# outputFile=r'/media/nkd/backup/5_lakemapping/iew100/predition_b5_max_iew50_4976.tif'
# vecter_path =outputFile.replace('tif','shp')
# field_name = "class"
# #            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
# ignore_values = [0]
# raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values